In [1]:
# Import the webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
# Import the chrome client
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
# Import pandas for data analysis
import pandas as pd
pd.set_option('display.max_columns', None)

In [8]:
players = {
    'Ja Morant': {'nbaid': '1629630', 'nbateamid': '1610612763', 'nbaseasons': ['2019-20','2020-21','2021-22'], 'bballrefid': 'm/moranja01', 'bballrefseasons': ['2020', '2021', '2022']},
    'Derrick Rose': {'nbaid': '201565', 'nbateamid': '1610612741', 'nbaseasons': ['2008-09','2009-10','2010-11'], 'bballrefid': 'r/rosede01', 'bballrefseasons': ['2009', '2010', '2011']},
    'Russell Westbrook': {'nbaid': '201566', 'nbateamid': '1610612760', 'nbaseasons': ['2008-09','2009-10','2010-11'], 'bballrefid': 'w/westbru01', 'bballrefseasons': ['2009', '2010', '2011']},
    'Allen Iverson': {'nbaid': '947', 'nbateamid': '1610612755', 'nbaseasons': ['1997-98','1998-99','1999-2000'], 'bballrefid': 'i/iversal01', 'bballrefseasons': ['1998', '1999', '2000']}
}

tables_NBA_100pos = ['//*[@id="__next"]/div[2]/section/div[4]/section[3]/div/div[2]/div[3]/table', '//*[@id="__next"]/div[2]/section/div[4]/section[3]/div/div[3]/div/div[3]/table', '//*[@id="__next"]/div[2]/section/div[4]/section[3]/div/div[4]/div/div[3]/table', '//*[@id="__next"]/div[2]/section/div[4]/section[3]/div/div[5]/div/div[3]/table', '//*[@id="__next"]/div[2]/section/div[4]/section[3]/div/div[6]/div/div[3]/table']

In [9]:
# Install web-driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# Getting data season average per 100pos
for key, value in players.items():
# Tell the web-driver what website to get.
    print('Starting with', key)
    driver.get('https://www.nba.com/stats/player/' + value['nbaid'] + '?PerMode=Per100Possessions')
    season_avg_per100 = pd.DataFrame()
    for j in range(5):
        print('Table', j+1)
        html_table = driver.find_element(By.XPATH, tables_NBA_100pos[j]).get_attribute("outerHTML") #This is the xpath (not full)
        table = pd.read_html(html_table)
        season_avg_per100 = pd.concat([season_avg_per100, table[0].tail(3).dropna(axis=1)], axis = 1)
    print('Season average per 100 possesions for', key, 'completed.')
    season_avg_per100.columns = season_avg_per100.columns.get_level_values(1)
    season_avg_per100 = season_avg_per100.loc[:,~season_avg_per100.columns.duplicated()]
    season_avg_per100.to_json('../data/'+ value['nbaid'] +'/season_avg_per100.json')
    for year in value['bballrefseasons']:
        season_boxscore = pd.DataFrame()
        driver.get('https://www.basketball-reference.com/players/' + value['bballrefid'] + '/gamelog/' + year)
        html_table = driver.find_element(By.XPATH, '//*[@id="pgl_basic"]').get_attribute('outerHTML')
        season_boxscore = pd.read_html(html_table)[0]
        season_boxscore = pd.read_html(html_table)
        season_boxscore[0] = season_boxscore[0].drop(['Rk', 'Age', 'Tm', 'Unnamed: 5', 'Opp', 'Unnamed: 7'], axis = 1).dropna()
        season_boxscore[0] = season_boxscore[0].set_index('G').drop('G', axis = 0)
        season_boxscore[0].to_json('../data/'+ value['nbaid'] +'/' + year + 'boxscores.json')
        print('Boxscore', key, year , 'completed.')




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Gonzalo\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


Starting with Allen Iverson
Table  1
Table  2
Table  3
Table  4
Table  5
Season average per 100 possesions for  Allen Iverson  completed.
Boxscore  Allen Iverson 1998  completed.
Boxscore  Allen Iverson 1999  completed.
Boxscore  Allen Iverson 2000  completed.
